In [20]:
import pandas as pd
import numpy as np
import random
import pickle
import sys
from keras.models import load_model
from keras import backend
import json 

sys.path.append("../../src/models/predict_model")
import predictor
sys.path.append("../../src/data")
import exclusionCriteria

## Get one record for a sample prediction

In [2]:
fileName = 'ED_TOTAL_2009_2009.csv'
with open('../../fileConfig.json') as config_file:    
        fileConfig = json.load(config_file)
fileLocation = fileConfig['dataDirectory'] + fileConfig['processedDirectory'] + fileName
cdc_file = pd.read_csv(fileLocation)    

In [3]:
# records with vital signs = -9 are filtered out for demo purposes
# todo: resolve records with missing data 
reload(exclusionCriteria)
cdc_file = exclusionCriteria.filter_out_records_na(cdc_file)

In [27]:
#getting one record from the input file, as an example
record_index = random.randint(0,len(cdc_file))
sample_record = cdc_file.iloc[[record_index]]
sample_record.head()

,VYEAR,VMONTH,VDAYR,AGE,ARRTIME,WAITTIME,LOV,RESIDNCE,SEX,ETHUN,...,Reason_Psychiatric_Complaint,Reason_Nervous_System,Reason_Cardiovascular_Other,Reason_Ears_Eyes_Complaint,Reason_Respiratory_Other,Reason_Gastrointestinal_Other,Reason_Genitourinary_Other,Reason_Skin_Hair_Nails_Complaint,Reason_Musculoskeletal_Other,Reason_Injury_Poisoning
5222,2009,4,2,87,926,-9,-9,1,1,2,...,0,1,0,0,0,0,0,0,1,0


## Thresholds and Model Configuration

In [40]:
fileESIThreshold_location = fileConfig['dataDirectory'] + fileConfig['resultDirectory'] + 'esi_thresholds.json'  
with open(fileESIThreshold_location) as config_file:    
        thresholdsConfig = json.load(config_file)
thresholdsConfig

{u'esi1_threshold': 0.8709694147109985,
 u'esi2_threshold': 0.7043555974960327,
 u'rss_threshold': 0.35}

In [55]:
with open('../../modelConfig.json') as config_file:    
        modelConfig = json.load(config_file)
        
modelConfig

{u'co_model': u'../../models/cdc_2009_nn_att_text_embedding.H5',
 u'max_text_length': u'../../models/cdc_2009_att_text_max_length.pickle',
 u'rss_model': u'../../models/cdc_rss_2009_nn_att_text_embedding.H5',
 u'tokenizer': u'../../models/cdc_2009_att_text_tokenizer.pickle'}

## Predict and pull attention relative weights for Model Interpretability

In [51]:
reload(predictor)
att_top = 15
co_pred, pred_rss, pred_esi, co_feature_importance, co_word_heat_map, rss_feature_importance, rss_word_heat_map = \
predictor.predict_and_get_att(modelConfig,thresholdsConfig,  att_top, sample_record)

Creating text for embeddings


In [53]:
print 'y_pred: ' , co_pred
co_feature_importance

y_pred:  [ 0.86773646]


,attention_weight,featureName,input_value
3,0.022630,chf_indicator,1
50,0.020733,Age_81_Above,1
1,0.020246,arrival_model,2: NOT by ambulance
41,0.020132,Reason_Musculoskeletal_Other,1
51,0.020112,reason1_for_visit,12201:
20,0.018343,Moderate_Tachypnea,1
52,0.018215,reason2_for_visit,19200: leg symptoms
34,0.015176,Reason_Nervous_System,1
27,0.013409,Reason_Shortness_of_Breath,0
5,0.013264,Age,87


In [54]:
word_heat_map

,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9,pos_10,pos_11,pos_12,pos_13
0,other,s,s,leg,symptoms,s,s,symptoms,of,the,face,nec,mass
1,0.00782466,0.00628352,0.00628352,0.0181018,0.00880685,0.00628352,0.00628352,0.00880685,0.0078089,0.00816092,0.00889456,0.00944738,0.00773853
2,,,,*,,,,,,,,*,
